<a href="https://colab.research.google.com/github/orin00/bucheon_Green/blob/main/faceModel_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요한 라이브러리 설치
# !pip install tensorflow opencv-python

In [1]:
# %cd /content/drive/MyDrive/face_yolov8/datasets/train/images

In [2]:
# !unzip 'images.zip'

In [3]:
# %cd /content/drive/MyDrive/face_yolov8/datasets/train/labels

In [4]:
# !unzip 'labels.zip'

In [13]:
# 이미지 파일명 가져오기
from glob import glob

train_img = glob('/content/drive/MyDrive/face_yolov8/datasets/train/images/*.jpg')
valid_img = glob('/content/drive/MyDrive/face_yolov8/datasets/valid/images/*.jpg')
train_labels = glob('/content/drive/MyDrive/face_yolov8/datasets/train/labels/*.txt')
valid_labels = glob('/content/drive/MyDrive/face_yolov8/datasets/valid/labels/*.txt')

print(len(train_img), len(train_labels), len(valid_img), len(valid_labels)) # 9016 9016 3864 3864

6975 6975 3002 3002


In [14]:
%cd /content/drive/MyDrive/face_yolov8/

/content/drive/MyDrive/face_yolov8


In [15]:
!pwd

/content/drive/MyDrive/face_yolov8


In [16]:
import tensorflow as tf
import os
from tensorflow.keras import layers, models

# 1. 경로 설정
try:
    train_image_folder = '/content/drive/MyDrive/face_yolov8/datasets/train/images'
    train_label_folder = '/content/drive/MyDrive/face_yolov8/datasets/train/labels'
    valid_image_folder = '/content/drive/MyDrive/face_yolov8/datasets/valid/images'
    valid_label_folder = '/content/drive/MyDrive/face_yolov8/datasets/valid/labels'
    print("Paths set successfully.")
except Exception as e:
    print(f"Error setting paths: {e}")

Paths set successfully.


In [17]:
# 2. 이미지 이름 가져오기
try:
    image_names_train = [f for f in os.listdir(train_image_folder) if f.endswith('.jpg')]
    image_names_valid = [f for f in os.listdir(valid_image_folder) if f.endswith('.jpg')]

    if not image_names_train:
        raise ValueError("No images found in the training folder.")
    if not image_names_valid:
        raise ValueError("No images found in the validation folder.")

    print("Image names loaded successfully.")
    print(f"Number of training images: {len(image_names_train)}")
    print(f"Number of validation images: {len(image_names_valid)}")

except Exception as e:
    print(f"Error loading image names: {e}")

# Tensor로 변환
try:
    image_names_train = tf.convert_to_tensor(image_names_train)
    image_names_valid = tf.convert_to_tensor(image_names_valid)
    print("Image names converted to tensor successfully.")
except Exception as e:
    print(f"Error converting image names to tensor: {e}")

Image names loaded successfully.
Number of training images: 6975
Number of validation images: 3002
Image names converted to tensor successfully.


In [18]:
# import os
# import tensorflow as tf

# max_bboxes = 10  # 바운딩 박스 최대 개수

# def delete_images_with_large_bboxes(image_names, image_folder, label_folder):
#     for image_name in image_names:
#         try:
#             # 이미지 경로 및 라벨 경로
#             image_path = os.path.join(image_folder, image_name.numpy().decode("utf-8"))
#             label_path = os.path.join(label_folder, image_name.numpy().decode("utf-8").replace('.jpg', '.txt'))

#             # 라벨 파일 읽기
#             bboxes = tf.io.read_file(label_path)
#             bboxes = tf.strings.split(bboxes, '\n')

#             # 유효한 바운딩 박스 개수 확인
#             bbox_count = 0
#             for bbox in bboxes:
#                 if bbox != '':
#                     bbox_count += 1

#             # 바운딩 박스가 10개 이상이면 이미지와 라벨 파일 삭제
#             if bbox_count > max_bboxes:
#                 os.remove(image_path)  # 이미지 삭제
#                 os.remove(label_path)  # 라벨 삭제
#                 print(f"Deleted image and label: {image_name.numpy().decode('utf-8')} (Bounding boxes: {bbox_count})")

#         except Exception as e:
#             print(f"Error processing {image_name.numpy().decode('utf-8')}: {e}")

# # 트레인 데이터에 적용
# delete_images_with_large_bboxes(image_names_train, train_image_folder, train_label_folder)

# # 검증 데이터에 적용
# delete_images_with_large_bboxes(image_names_valid, valid_image_folder, valid_label_folder)

KeyboardInterrupt: 

In [19]:
print(len(image_names_train))
print(len(image_names_valid))

6975
3002


In [20]:
# 3. 데이터셋 로드
try:
    train_dataset = tf.data.Dataset.from_tensor_slices(image_names_train)
    valid_dataset = tf.data.Dataset.from_tensor_slices(image_names_valid)
    print("Datasets created successfully.")
except Exception as e:
    print(f"Error creating datasets: {e}")

Datasets created successfully.


In [ ]:
import tensorflow as tf
import os

# 4. 데이터셋 맵핑 및 이미지 및 라벨 로드
train_images = []
train_bboxes = []

max_boxes = 10  # 최대 바운딩 박스 수

# 데이터 로딩 최적화
try:
    for image_name in image_names_train:
        try:
            image_path = os.path.join(train_image_folder, image_name.numpy().decode("utf-8"))
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, [224, 224]) / 255.0  # 정규화와 크기 조정을 한 줄로

            label_path = os.path.join(train_label_folder, image_name.numpy().decode("utf-8").replace('.jpg', '.txt'))
            bboxes = tf.io.read_file(label_path)
            bboxes = tf.strings.split(bboxes, '\n')

            bbox_list = []
            for bbox in bboxes:
              if bbox != '':
                  values = tf.strings.to_number(tf.strings.split(bbox, ' '))
                  # YOLO 포맷에서 바운딩 박스 좌표 변환 + 클래스 ID 추가
                  class_id = values[0]  # 첫 번째 값은 클래스 ID
                  x_center, y_center, width, height = values[1] * 224, values[2] * 224, values[3] * 224, values[4] * 224
                  bbox_list.append([
                      x_center - (width / 2), y_center - (height / 2),
                      x_center + (width / 2), y_center + (height / 2),
                      class_id  # 클래스 ID를 추가
                  ])

            # 바운딩 박스 개수 제한
            bbox_list = bbox_list[:max_boxes]

            if bbox_list:
                bbox_tensor = tf.convert_to_tensor(bbox_list, dtype=tf.float32)
                # 패딩 추가
                padding_size = max_boxes - tf.shape(bbox_tensor)[0]
                if padding_size > 0:
                    bbox_tensor = tf.pad(bbox_tensor, [[0, padding_size], [0, 0]], constant_values=0)
            else:
                bbox_tensor = tf.zeros((max_boxes, 5), dtype=tf.float32)  # 클래스 ID를 포함하여 (max_boxes, 5)

            image.set_shape([224, 224, 3])
            bbox_tensor.set_shape([max_boxes, 5])  # 5차원으로 고정

            train_images.append(image)
            train_bboxes.append(bbox_tensor)

            print(f"Loaded image and label for: {image_name.numpy().decode('utf-8')}")
        except Exception as e:
            print(f"Error loading image and label for {image_name.numpy().decode('utf-8')}: {e}")

except Exception as e:
    print(f"Error processing training images and labels: {e}")

# Tensor로 변환
try:
    train_images = tf.convert_to_tensor(train_images)
    train_bboxes = tf.convert_to_tensor(train_bboxes)
    print("Training images and bboxes converted to tensor successfully.")
except Exception as e:
    print(f"Error converting training images and bboxes to tensor: {e}")


Loaded image and label for: wider_5330.jpg
Loaded image and label for: wider_9576.jpg
Loaded image and label for: wider_5336.jpg
Loaded image and label for: wider_4594.jpg
Loaded image and label for: wider_10232.jpg
Loaded image and label for: wider_6807.jpg
Loaded image and label for: wider_9843.jpg
Loaded image and label for: wider_10706.jpg
Loaded image and label for: wider_10570.jpg
Loaded image and label for: wider_7016.jpg
Loaded image and label for: wider_9215.jpg
Loaded image and label for: wider_9657.jpg
Loaded image and label for: wider_9955.jpg
Loaded image and label for: wider_7658.jpg
Loaded image and label for: wider_12079.jpg
Loaded image and label for: wider_10197.jpg
Loaded image and label for: wider_461.jpg
Loaded image and label for: wider_1852.jpg
Loaded image and label for: wider_10771.jpg
Loaded image and label for: wider_3205.jpg
Loaded image and label for: wider_11199.jpg
Loaded image and label for: wider_9695.jpg
Loaded image and label for: wider_11356.jpg
Load

In [ ]:
# 검증 데이터셋도 동일하게 처리
valid_images = []
valid_bboxes = []

# 최대 바운딩 박스 수
max_boxes = 10

try:
    for image_name in image_names_valid:
        try:
            image_path = os.path.join(valid_image_folder, image_name.numpy().decode("utf-8"))
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, [224, 224]) / 255.0  # 정규화와 크기 조정을 한 줄로

            label_path = os.path.join(valid_label_folder, image_name.numpy().decode("utf-8").replace('.jpg', '.txt'))
            bboxes = tf.io.read_file(label_path)
            bboxes = tf.strings.split(bboxes, '\n')

            bbox_list = []
            for bbox in bboxes:
              if bbox != '':
                  values = tf.strings.to_number(tf.strings.split(bbox, ' '))
                  # YOLO 포맷에서 바운딩 박스 좌표 변환 + 클래스 ID 추가
                  class_id = values[0]  # 첫 번째 값은 클래스 ID
                  x_center, y_center, width, height = values[1] * 224, values[2] * 224, values[3] * 224, values[4] * 224
                  bbox_list.append([
                      x_center - (width / 2), y_center - (height / 2),
                      x_center + (width / 2), y_center + (height / 2),
                      class_id  # 클래스 ID를 추가
                  ])

            # 바운딩 박스가 10개보다 적으면 패딩 추가
            if bbox_list:
                bbox_tensor = tf.convert_to_tensor(bbox_list, dtype=tf.float32)
                # 패딩 추가
                padding_size = max_boxes - tf.shape(bbox_tensor)[0]
                if padding_size > 0:
                    bbox_tensor = tf.pad(bbox_tensor, [[0, padding_size], [0, 0]], constant_values=0)
            else:
                bbox_tensor = tf.zeros((max_boxes, 5), dtype=tf.float32)  # 클래스 ID를 포함하여 (max_boxes, 5)

            image.set_shape([224, 224, 3])
            bbox_tensor.set_shape([max_boxes, 5])  # 5차원으로 고정

            valid_images.append(image)
            valid_bboxes.append(bbox_tensor)

            print(f"Loaded validation image and label for: {image_name.numpy().decode('utf-8')}")
        except Exception as e:
            print(f"Error loading validation image and label for {image_name.numpy().decode('utf-8')}: {e}")

except Exception as e:
    print(f"Error processing validation images and labels: {e}")

# Tensor로 변환
try:
    valid_images = tf.convert_to_tensor(valid_images)
    valid_bboxes = tf.convert_to_tensor(valid_bboxes)
    print("Validation images and bboxes converted to tensor successfully.")
except Exception as e:
    print(f"Error converting validation images and bboxes to tensor: {e}")


Loaded validation image and label for: wider_0.jpg
Loaded validation image and label for: wider_10050.jpg
Loaded validation image and label for: wider_10063.jpg
Loaded validation image and label for: wider_10003.jpg
Loaded validation image and label for: wider_10028.jpg
Loaded validation image and label for: wider_10086.jpg
Loaded validation image and label for: wider_10128.jpg
Loaded validation image and label for: wider_10122.jpg
Loaded validation image and label for: wider_10067.jpg
Loaded validation image and label for: wider_10116.jpg
Loaded validation image and label for: wider_10119.jpg
Loaded validation image and label for: wider_10134.jpg
Loaded validation image and label for: wider_10178.jpg
Loaded validation image and label for: wider_10133.jpg
Loaded validation image and label for: wider_1014.jpg
Loaded validation image and label for: wider_10140.jpg
Loaded validation image and label for: wider_1025.jpg
Loaded validation image and label for: wider_10217.jpg
Loaded validatio

In [ ]:
# 5. 데이터셋 생성
try:
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_bboxes)).batch(16).prefetch(tf.data.experimental.AUTOTUNE)
    valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images, valid_bboxes)).batch(16).prefetch(tf.data.experimental.AUTOTUNE)
    print("Batched datasets created successfully.")
except Exception as e:
    print(f"Error creating batched datasets: {e}")


Batched datasets created successfully.


In [ ]:
from tensorflow.keras import layers, models

# YOLO 모델 생성 (출력 차원 수정)
input_shape = (224, 224, 3)
num_classes = 1  # 감지할 클래스 수
num_bboxes = 10  # 바운딩 박스 개수를 10으로 설정

try:
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)

    output_size = num_bboxes * (4 + num_classes)  # 바운딩 박스 좌표 4개 + 클래스 ID 1개
    outputs = layers.Dense(output_size)(x)
    outputs = layers.Reshape((num_bboxes, 4 + num_classes))(outputs)  # (10, 5) 형태로 조정


    model = models.Model(inputs=inputs, outputs=outputs)
    print("Model created successfully.")
except Exception as e:
    print(f"Error creating the model: {e}")


Model created successfully.


In [ ]:
import tensorflow as tf

# 7. 모델 컴파일
try:
    def custom_loss(y_true, y_pred):
      num_bboxes = 10  # 바운딩 박스 개수
      num_classes = 1  # 클래스 수

      # 바운딩 박스 좌표와 클래스 정보 분리
      y_true_bboxes = y_true[:, :, :4]  # 바운딩 박스 좌표 (10, 4)
      y_true_classes = y_true[:, :, 4]  # 클래스 ID (10, 1)

      y_pred_bboxes = y_pred[:, :, :4]  # 바운딩 박스 좌표 (10, 4)
      y_pred_classes = y_pred[:, :, 4]  # 클래스 ID (10, 1)

      # 바운딩 박스 손실 (MSE)
      bbox_loss = tf.reduce_mean(tf.square(y_true_bboxes - y_pred_bboxes))

      # 클래스 손실 (Binary Crossentropy)
      class_loss = tf.keras.losses.binary_crossentropy(y_true_classes, y_pred_classes)

      # 총 손실
      return bbox_loss + class_loss

    model.compile(optimizer='adam', loss=custom_loss)
    print("Model compiled successfully.")
except Exception as e:
    print(f"Error compiling the model: {e}")


Model compiled successfully.


In [ ]:
# 8. 모델 학습
try:
    model.fit(train_dataset, validation_data=valid_dataset, epochs=100, verbose=1)
    print("Model training completed successfully.")
except Exception as e:
    print(f"Error during model training: {e}")


Epoch 1/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 601s 3s/step - loss: 3356.8445 - val_loss: 2134.0693
Epoch 2/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 613s 3s/step - loss: 2843.3352 - val_loss: 2297.0488
Epoch 3/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 617s 3s/step - loss: 2602.3733 - val_loss: 2023.1844
Epoch 4/50
 44/228 ━━━━━━━━━━━━━━━━━━━━ 7:24 2s/step - loss: 2482.4373

KeyboardInterrupt: 

In [ ]:
# 모델 저장 경로
model.save('/content/drive/MyDrive/face_yolov8/yolo_face_detection_model.h5')